In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# data acquisition
df = pd.read_csv('/content/USDT-BTC.csv')
df['Date'] = pd.to_datetime(df['Date'])

# data selection

X = df[['Open', 'High', 'Low', 'Adj Close', 'Volume']]
y = df['Close']

# dropping null values
missing_values = y.isnull()
X = X[~missing_values]
y = y.dropna()

# splitting, training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [ ]:
from sklearn.linear_model import Ridge

# Train Ridge Regression model
ridge_model = Ridge(alpha=1.0)  # You can adjust the alpha parameter for regularization
ridge_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = ridge_model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")


Mean Squared Error (MSE): 4.5300807773669455e-13
R-squared (R2): 0.9999184455618215


In [ ]:
daily_returns = pd.Series(y_pred).pct_change()
cumulative_returns = (1 + daily_returns).cumprod()
drawdowns = 1 - (cumulative_returns / cumulative_returns.cummax())
max_drawdown = drawdowns.max()
# Calculate daily returns
daily_returns = pd.Series(y_pred).pct_change()

# Calculate cumulative returns
cumulative_returns = (1 + daily_returns).cumprod()

# Calculate drawdowns
drawdowns = 1 - (cumulative_returns / cumulative_returns.cummax())

# Find maximum drawdown
max_drawdown = drawdowns.max()

print(f"Maximum Drawdown: {max_drawdown}")


Maximum Drawdown: 0.9480664489696534


In [ ]:
# Calculate daily returns
predicted_returns = (y_pred[1:] - y_pred[:-1]) / y_pred[:-1]
actual_returns = (y_test.values[1:] - y_test.values[:-1]) / y_test.values[:-1]

# Assume risk-free rate is 0.02 (2%)
risk_free_rate = 0.02

# Calculate excess returns
excess_returns = predicted_returns - risk_free_rate

# Calculate Sharpe ratio
sharpe_ratio = excess_returns.mean() / excess_returns.std()

print("Sharpe Ratio of the model : ",sharpe_ratio)


Sharpe Ratio of the model :  0.37673869068805604


In [ ]:
# Assuming y_pred contains the predicted returns
# Define a simple trading strategy
threshold = 0.5  # Arbitrary threshold for trading decisions
profits = []

for i in range(len(y_pred)):
    if y_pred[i] > threshold:  # Buy signal
        profits.append(y_test.iloc[i + 1] - y_test.iloc[i])  # Profit is the change in price

# Calculate net profit exceeding return
net_profit = sum(profits)
baseline_return = y_test.iloc[-1] - y_test.iloc[0]  # Buy-and-hold return
exceeding_return = net_profit - baseline_return

print(f"Net Profit Exceeding Return: {exceeding_return}")


Net Profit Exceeding Return: 7e-05


In [ ]:
risk = mse  # Considering MSE as the risk
reward = r2  # Considering R-squared as the reward

if risk != 0:  # To avoid division by zero
    risk_reward_ratio = reward / risk
    print(f"Risk-Reward Ratio: {risk_reward_ratio}")
else:
    print("Risk-Reward Ratio cannot be computed because risk is zero.")


Risk-Reward Ratio: 2207286127341.447


In [ ]:
# Assuming 'df' is the original DataFrame with 'Date' column in datetime format

# Sort the DataFrame by 'Date'
df_sorted = df.sort_values('Date')

# Find the maximum duration time of a single trade
max_duration = max(df_sorted['Date'].diff().dt.total_seconds().dropna())

print(f"Maximum duration time of a single trade: {max_duration} seconds")


Maximum duration time of a single trade: 2678400.0 seconds
